In [116]:
import yfinance as yf
import pandas as pd

In [117]:
tickers = ["NTPC.NS",
"APOLLOHOSP.NS",
"CIPLA.NS",
"BRITANNIA.NS",
"ADANIENT.NS",
"ULTRACEMCO.NS",
"HEROMOTOCO.NS",
"WIPRO.NS",
"MARUTI.NS",
"BHARTIARTL.NS",
"INDUSINDBK.NS",
"TITAN.NS",
"COALINDIA.NS",
"KOTAKBANK.NS",
"ONGC.NS",
"LTIM.NS",
"LT.NS",
"TCS.NS",
"NESTLEIND.NS",
"HDFCLIFE.NS",
"TECHM.NS",
"ITC.NS",
"RELIANCE.NS",
"BAJFINANCE.NS",
"BAJAJFINSV.NS",
"HINDALCO.NS",
"BAJAJ-AUTO.NS",
"TATACONSUM.NS",
"TATASTEEL.NS",
"EICHERMOT.NS",
"GRASIM.NS",
"HINDUNILVR.NS",
"M&M.NS",
"TATAMOTORS.NS",
"WIPRO.NS",
"APOLLOHOSP.NS",    
"DRREDDY.NS",
"TITAN.NS",
"SBIN.NS",
"BPCL.NS",
"KOTAKBANK.NS",
"UPL.NS",
"INFY.NS",
"SUNPHARMA.NS",
"JSWSTEEL.NS",
"HDFCBANK.NS",
"AXISBANK.NS",
"HCLTECH.NS",
"LTIM.NS",
"DIVISLAB.NS",
"ADANIPORTS.NS",
"SBILIFE.NS",
"ULTRACEMCO.NS",
]

In [118]:
df = yf.download(tickers,period='2y',interval='1d')

[*********************100%%**********************]  47 of 47 completed


In [119]:
df.to_csv('NIFTY50_LATEST.CSV')

PermissionError: [Errno 13] Permission denied: 'NIFTY50_LATEST.CSV'

In [120]:
df = pd.read_csv('NIFTY50_LATEST.CSV')

In [121]:
# Filter columns to keep 'Date' and those that contain the word 'Adj Close'
columns_to_keep = df.columns[df.columns.str.contains('Adj Close') | df.columns.str.contains('Date')]

# Select only the columns to keep
df_filtered = df[columns_to_keep]

# Reset the column names to default integer indices
df_filtered.columns = df_filtered.iloc[0]

# Print the filtered DataFrame without headers
print(df_filtered)

0        Ticker        ADANIENT.NS       ADANIPORTS.NS      APOLLOHOSP.NS  \
0        Ticker        ADANIENT.NS       ADANIPORTS.NS      APOLLOHOSP.NS   
1          Date                NaN                 NaN                NaN   
2    2022-05-17    2182.8310546875   734.6871337890625  3787.212646484375   
3    2022-05-18  2179.084716796875   749.5337524414062  3709.594482421875   
4    2022-05-19  2112.795654296875   722.8985595703125  3607.096435546875   
..          ...                ...                 ...                ...   
493  2024-05-14  3037.550048828125             1331.25    5862.7998046875   
494  2024-05-15            3049.75  1337.8499755859375    5848.7998046875   
495  2024-05-16   3041.64990234375   1345.050048828125    5931.7001953125   
496  2024-05-17             3060.5   1335.699951171875    5953.0498046875   
497  2024-05-18             3061.0              1345.0             5955.0   

0           AXISBANK.NS     BAJAJ-AUTO.NS       BAJAJFINSV.NS  \
0         

In [122]:
# Drop the first row
df_filtered = df_filtered.drop(index=0)

# Replace NaN values in the new first row with the headers
df_filtered.iloc[0, 1:] = df_filtered.columns[1:]

# Rename the first column to 'Date'
df_filtered.columns = ['Date'] + df_filtered.columns[1:].tolist()

# Drop the second row (which now contains the old headers)
df_filtered = df_filtered.drop(index=1)

# Reset the index to use 'Date'
df_filtered['Date'] = pd.to_datetime(df_filtered['Date'])
df_filtered = df_filtered.set_index('Date')

# Print the filtered DataFrame to verify
print(df_filtered)


                  ADANIENT.NS       ADANIPORTS.NS      APOLLOHOSP.NS  \
Date                                                                   
2022-05-17    2182.8310546875   734.6871337890625  3787.212646484375   
2022-05-18  2179.084716796875   749.5337524414062  3709.594482421875   
2022-05-19  2112.795654296875   722.8985595703125  3607.096435546875   
2022-05-20    2217.4990234375   757.1790161132812   3643.05029296875   
2022-05-23   2168.64404296875   743.3682250976562    3691.6669921875   
...                       ...                 ...                ...   
2024-05-14  3037.550048828125             1331.25    5862.7998046875   
2024-05-15            3049.75  1337.8499755859375    5848.7998046875   
2024-05-16   3041.64990234375   1345.050048828125    5931.7001953125   
2024-05-17             3060.5   1335.699951171875    5953.0498046875   
2024-05-18             3061.0              1345.0             5955.0   

                   AXISBANK.NS      BAJAJ-AUTO.NS       BAJAJFI

In [123]:
# Ensure all relevant columns are numeric and convert to float
df_filtered = df_filtered.apply(pd.to_numeric, errors='coerce').astype(float)

In [124]:
assets = df_filtered.columns

In [125]:
#Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [126]:
# Calculate the expected returns and the sample covariance matrix
mu = expected_returns.mean_historical_return(df_filtered)
S= risk_models.sample_cov(df_filtered)

In [127]:
#Optimize for maximmal Sharpe Ratio
ef = EfficientFrontier(mu,S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
# Sort the cleaned weights by value in descending order
sorted_weights = {k: v for k, v in sorted(cleaned_weights.items(), key=lambda item: item[1], reverse=True)}

# Print the cleaned weights with asset names and values (only where weight > 0)
print("Optimized Portfolio Weights:")
for asset, weight in sorted_weights.items():
    if weight > 0:
        print(f"{asset}: {weight:.4f}")

# Print the portfolio performance
print("\nPortfolio Performance:")
ef.portfolio_performance(verbose=True)

Optimized Portfolio Weights:
BAJAJ-AUTO.NS: 0.2040
COALINDIA.NS: 0.1487
M&M.NS: 0.1211
NTPC.NS: 0.1198
SUNPHARMA.NS: 0.1024
BHARTIARTL.NS: 0.1002
ITC.NS: 0.0810
LT.NS: 0.0576
BRITANNIA.NS: 0.0504
CIPLA.NS: 0.0111
NESTLEIND.NS: 0.0038

Portfolio Performance:
Expected annual return: 54.0%
Annual volatility: 12.5%
Sharpe Ratio: 4.16


(0.5398547315971309, 0.12495794599073021, 4.1602374901048345)

In [128]:
# Get the latest prices, ensure it's a Series with numeric values
latest_prices = get_latest_prices(df_filtered)

# Ensure latest_prices is a pandas Series with no NaNs
latest_prices = latest_prices.dropna().astype(float)

# Define portfolio value
portfolio_val = 500000

# Discrete allocation
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_val)
allocation, leftover = da.lp_portfolio()

# Print the discrete allocation and funds remaining
print('Discrete allocation:', allocation)
print('Funds remaining: Rs', leftover)

Discrete allocation: {'BAJAJ-AUTO.NS': 12, 'BHARTIARTL.NS': 37, 'BRITANNIA.NS': 5, 'CIPLA.NS': 4, 'COALINDIA.NS': 158, 'ITC.NS': 93, 'LT.NS': 8, 'M&M.NS': 24, 'NTPC.NS': 163, 'SUNPHARMA.NS': 33}
Funds remaining: Rs 278.0449465744896
